In [2]:
import sys
sys.path.append("C:/Users/Haziq/OneDrive - Imperial College London/Third Year/Fetal VR Group Project/Files/Done files/SegmentationDev/4.notebook/notebookretry")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
%run dataset.ipynb

In [4]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = torch.flatten(inputs)
        targets = torch.flatten(targets)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1-dice

In [43]:
class Dice(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(Dice, self).__init__()

    def forward(self, inputs, targets, smooth=0):

        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)
        
        dinputs = torch.argmax(inputs, dim=1)

        d1 = dice_coefficient(dinputs, targets, 1)
        d2 = dice_coefficient(dinputs, targets, 2)
        
        dice = (d1+d2)/2

        return dice       


In [5]:
class Dicebleh(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(Dice, self).__init__()

    def forward(self, inputs, targets, smooth=0):

        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)

        
        inputs = torch.argmax(inputs, dim=1)
        # print(targets.size())

        #flatten label and prediction tensors
        inputs = torch.flatten(inputs)
        targets = torch.flatten(targets)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return dice

In [6]:
#Test the loss with 2 images to see if im getting the right values

In [ ]:
def dice_coefficient(y_true, y_pred, class_index):
    # print(y_true == class_index)
    intersection = torch.sum((y_true == class_index) & (y_pred == class_index))
    union = torch.sum(y_true == class_index) + torch.sum(y_pred == class_index)
    dice = (2. * intersection) / (union + 1e-9)  # Adding a small epsilon to avoid division by zero
    return dice

In [7]:
class DiceCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        batchnum = targets.size(dim=0)
        #print(targets.reshape(batchnum,112,112,112).size())

        #flatten label and prediction tensors
        #Doing it for just 1 and 2

        dinputs = torch.argmax(inputs, dim=1)

        d1 = dice_coefficient(dinputs, targets, 1)
        # print(d1)
        d2 = dice_coefficient(dinputs, targets, 2)
        # print(d2)
        
        dice_loss = 1-((d1+d2)/2)
        
        
        CE = F.cross_entropy(inputs, targets.reshape(batchnum,112,112,112).long(), reduction="mean")
        Dice_CE = CE + dice_loss

        # print("CE:", CE)
        # print("DICE:", dice_loss)
        # print()

        #Ignore the mask having 0 intensity
        #Only consider level 1 and level 2
        return Dice_CE

In [8]:
class DiceCEMultiClassLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceCEMultiClassLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        """The inputs are already being softmaxed"""
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        #inputs = inputs.view(-1)
        #targets = targets.view(-1)
        # loss = F.cross_entropy()
            
        #Ignoring the background
        numchan = inputs.size(dim=1)

        # print(inputs.unique)
        # print(targets.unique)

        TotDice_CE = 0

        for channel in range(numchan):
            input_ = inputs[:,channel,:,:,:].resize(1,1,112,112,112)
            target = targets[:,channel,:,:,:].resize(1,1,112,112,112)

            intersection = (input_ * target).sum()
            dice_loss = 1 - (2.*intersection + smooth)/(input_.sum() + target.sum() + smooth)
            
            CELOSS = F.binary_cross_entropy(input_, target, reduction="mean").item()
            Dice_CE = CELOSS + dice_loss

            # print(input_.size())

            # print("chan: ", channel)
            # print("CE Loss:", CELOSS)
            # print("Dice Loss:", dice_loss)
            TotDice_CE += Dice_CE

        TotDice_CE = TotDice_CE/numchan
        # print(TotDice_CE)

        return TotDice_CE

In [9]:
class CELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(CELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        loss = F.cross_entropy(inputs, targets)

        return loss

In [41]:


def dice_score_single_class(y_true, y_pred, class_index):
    dice = dice_coefficient(y_true, y_pred, class_index)
    return dice

In [42]:
def main():
    import matplotlib.pyplot as plt
    y = readNifti_mask("/home/yiho/Haziq/3D 112 Dataset/test/mask/Case_122_Pre/Case_122_Pre_mask_time002.nii")
    #Calculating Dice
    
    intersection = (y*y).sum()
    dice = (2.*intersection)/(y.sum() + y.sum())
    # print("Dice", dice)

    dice = dice_score_single_class(y, y, 2)
    # print("Dice multiclass score:", dice)

if __name__ == "__main__":
    pass
    # main()

Dice tensor(1.6371)
tensor([[[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],

         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],

         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,


    y_softmax = F.softmax(y)

    nnloss =  nn.CrossEntropyLoss()
    loss = F.cross_entropy(y_softmax, y_softmax, reduction="mean")
    nnl = nnloss(y,y).item()
    print("Cross Entropy Loss : ", loss)
    print("nn loss: ", nnl)
    
    #Calculating Dice
    intersection = (y*y).sum()
    dice_loss = 1 - (2.*intersection + 1)/(y.sum() + y.sum() + 1)
    print("Dice Loss :", dice_loss)

    # Generate identical matrices as input and target
    input_matrix = torch.randn(10000, 5)  # Example input matrix
    target_matrix = torch.randn(10000, 5)  # Example target matrix

    # Compute softmax on the input matrix (assuming the raw output of your model)
    input_softmax = F.softmax(input_matrix, dim=1)

    # Compute cross-entropy loss
    loss = F.cross_entropy(input_softmax, input_softmax)

    print(loss.item())  # Print the loss value